# Finding Cu crystal ground state via the Augmented Plane method

Adapted from Thijssen

In [1]:
import numpy as np
from scipy import special as sp

In [2]:
# constants
a = 6.822 #au
R = 1
V = 3*(a**3)/4

l_max = 3
nLim = 3 

b1 = 2*np.pi*np.array([-1,1,1])/a
b2 = 2*np.pi*np.array([1,-1,1])/a
b3 = 2*np.pi*np.array([1,1,-1])/a
Ks =[l*b1 + m*b2 + n*b3 for l in range(-nLim,nLim+1) for m in range(-nLim,nLim+1) for n in range(-nLim,nLim+1)]



In [3]:
#generate A
A = np.empty([len(Ks),len(Ks)])
for i,K_i in enumerate(Ks):
    for j,K_j in enumerate(Ks):
        kDiff = np.linalg.norm(K_i-K_j)
        if kDiff == 0:
            A[i,j] = 1
        else:
            A[i,j] = -4*np.pi*(R**2)*sp.spherical_jn(1,kDiff*R)/(V*kDiff)

print(A)




[[ 1.00000000e+00 -1.35026863e-02 -5.09872972e-03 ... -6.24728351e-04
  -6.24843489e-04 -5.61053190e-04]
 [-1.35026863e-02  1.00000000e+00 -1.35026863e-02 ... -6.24843489e-04
  -6.24728351e-04 -6.24843489e-04]
 [-5.09872972e-03 -1.35026863e-02  1.00000000e+00 ... -5.61053190e-04
  -6.24843489e-04 -6.24728351e-04]
 ...
 [-6.24728351e-04 -6.24843489e-04 -5.61053190e-04 ...  1.00000000e+00
  -1.35026863e-02 -5.09872972e-03]
 [-6.24843489e-04 -6.24728351e-04 -6.24843489e-04 ... -1.35026863e-02
   1.00000000e+00 -1.35026863e-02]
 [-5.61053190e-04 -6.24843489e-04 -6.24728351e-04 ... -5.09872972e-03
  -1.35026863e-02  1.00000000e+00]]


In [4]:
#shall use hydrogen atom approximation as all other electrons have a shielding effect (this is rough)

def iterateRK4(t_n,y_n,h,f):
    """
    Iterates by RK4 method.
    """
    k_1 = f(t_n, y_n)
    k_2 = f(t_n + h/2, y_n + h*k_1/2)
    k_3 = f(t_n + h/2, y_n + h*k_2/2)
    k_4 = f(t_n + h, y_n + h*k_3)

    return y_n + h*(k_1 + 2*k_2 + 2*k_3 + k_4)/6

def intialCond(l,r_0):
    #from small r solution to radial schrodinger eq, remember its u not R
    return np.array([r_0**(l+1),(l+1)*r_0**l])

def getfRadial(l,E):
    def f(r,y):
        u = y[0]
        p = y[1] #p = du/dr
        f_u = p
        f_p = (l*(l+1)/(r**2) - 2*E - 2/r)*u
        return np.array([f_u,f_p])
    return f
        
r_0 = 0.1
h = 0.05
def getRatio(l,E):
    rs = [r_0]
    ys = [intialCond(l,r_0)]
    for i in range(int((R - r_0)/h)):
        rs.append(rs[-1]+h)
        ys.append(iterateRK4(rs[-1],ys[-1],h,getfRadial(l,E)))
    
    yBoundary = ys[-1]
    return yBoundary[1]/yBoundary[0] - 1/R

In [8]:
def findDet(k,E):
    #set up k dependent matricies 
    B = np.empty([len(Ks),len(Ks)])
    C = np.empty([len(Ks),len(Ks),len(Ks)])
    qs = [k + K for K in Ks]

    #find B and C
    for i in range(len(Ks)):
        for j in range(len(Ks)):
            qDot = np.dot(qs[i],qs[j])
            B[i,j] = A[i,j]*qDot/2
            qi = np.linalg.norm(qs[i])
            qj = np.linalg.norm(qs[j])
            for l in range(l_max): 
                if qi == 0 or qj == 0:
                    C[i,j,l] = 0
                else: 
                    C[i,j,l] = (2*l + 1)*2*np.pi*(R**2)*sp.eval_legendre(l,qDot/(qi*qj))*sp.spherical_jn(l,qi*R)*sp.spherical_jn(l,qj*R)/V
    
    H = -E*A + B
    print(H)
    for l in range(l_max):
        H += C[:,:,l]*getRatio(l,E)
    
    return np.linalg.det(H - E*np.identity(len(Ks)))


In [9]:
ks = [2*np.pi*np.array([i/10,0,0])/a for i in range(10)]
EMesh  = np.arange(-0.1,0.2)

for k in ks:
    detOfE = np.vectorize(findDet)(k,EMesh)
    print(detOfE)

[[ 1.15517030e+01 -1.38798049e-01 -4.59236472e-02 ...  5.50190768e-03
   6.29797969e-03  6.36890917e-03]
 [-1.38798049e-01  1.02792915e+01 -1.38798049e-01 ...  6.29797969e-03
   6.29681918e-03  6.29797969e-03]
 [-4.59236472e-02 -1.38798049e-01  1.15517030e+01 ...  6.36890917e-03
   6.29797969e-03  5.50190768e-03]
 ...
 [ 5.50190768e-03  6.29797969e-03  6.36890917e-03 ...  1.15517030e+01
  -1.38798049e-01 -4.59236472e-02]
 [ 6.29797969e-03  6.29681918e-03  6.29797969e-03 ... -1.38798049e-01
   1.02792915e+01 -1.38798049e-01]
 [ 6.36890917e-03  6.29797969e-03  5.50190768e-03 ... -4.59236472e-02
  -1.38798049e-01  1.15517030e+01]]


c:\workspace\git-repos\physics-projects\.venv\lib\site-packages\numpy\linalg\linalg.py:2139: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


[[ 1.15517030e+01 -1.38798049e-01 -4.59236472e-02 ...  5.50190768e-03
   6.29797969e-03  6.36890917e-03]
 [-1.38798049e-01  1.02792915e+01 -1.38798049e-01 ...  6.29797969e-03
   6.29681918e-03  6.29797969e-03]
 [-4.59236472e-02 -1.38798049e-01  1.15517030e+01 ...  6.36890917e-03
   6.29797969e-03  5.50190768e-03]
 ...
 [ 5.50190768e-03  6.29797969e-03  6.36890917e-03 ...  1.15517030e+01
  -1.38798049e-01 -4.59236472e-02]
 [ 6.29797969e-03  6.29681918e-03  6.29797969e-03 ... -1.38798049e-01
   1.02792915e+01 -1.38798049e-01]
 [ 6.36890917e-03  6.29797969e-03  5.50190768e-03 ... -4.59236472e-02
  -1.38798049e-01  1.15517030e+01]]
[[ 1.15517030e+01 -1.38798049e-01 -4.59236472e-02 ...  5.50190768e-03
   6.29797969e-03  6.36890917e-03]
 [-1.38798049e-01  1.02792915e+01 -1.38798049e-01 ...  6.29797969e-03
   6.29681918e-03  6.29797969e-03]
 [-4.59236472e-02 -1.38798049e-01  1.15517030e+01 ...  6.36890917e-03
   6.29797969e-03  5.50190768e-03]
 ...
 [ 5.50190768e-03  6.29797969e-03  6.3689091

KeyboardInterrupt: 